# Part 4: Using GPU acceleration with PyTorch

In [2]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 KB 10.1 MB/s eta 0:00:00


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [3]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 8.9922e-01, -4.8753e-02, -8.4552e-02,  ..., -6.6615e-01,
          1.4415e+00, -8.2414e-01],
        [ 4.9537e-01, -1.3525e+00, -5.4746e-02,  ..., -5.7113e-02,
         -1.4739e+00,  5.1458e-01],
        [-2.5751e-01,  8.1089e-02,  1.7258e-01,  ..., -5.8627e-02,
          6.6381e+00, -3.7224e-01],
        ...,
        [ 1.9261e+00, -1.4838e+00,  2.7227e-01,  ...,  1.2913e-01,
          1.0493e-03,  1.6183e-01],
        [ 3.0941e-01, -5.6237e-01,  2.3231e-01,  ..., -9.0475e-01,
         -1.1124e+00, -2.3206e-01],
        [-6.4088e-03,  1.5481e+00,  1.9001e+00,  ...,  4.7840e-02,
          2.0447e-02,  9.7082e-01]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [4]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-2.3716e-02, -1.3564e-01,  6.8965e-01,  ...,  2.4877e-02,
         -2.0724e-03, -5.9453e-01],
        [ 9.2110e-01, -6.7873e-01,  4.1470e-01,  ..., -1.3326e-01,
         -3.0942e-02, -2.8554e-02],
        [ 3.6949e-01,  1.0093e+00, -2.5480e-02,  ..., -1.4989e+00,
         -8.5534e-01,  8.4100e-01],
        ...,
        [-8.3488e-01, -6.1579e-02, -1.7796e-01,  ...,  1.0754e-01,
          1.1085e-01,  8.6781e-01],
        [ 4.3666e-01,  4.0459e-03,  3.4066e-01,  ..., -4.1809e-01,
          8.6325e-01, -9.1001e-01],
        [-1.4137e-01,  3.3794e+00,  2.6813e-02,  ...,  1.0863e+00,
          4.9083e-01, -1.4126e-01]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[ 1.0989e-01,  1.0113e+00, -6.2610e-02,  ..., -2.4303e-01,
          7.8832e-03, -1.3991e-01],
        [ 1.4091e+00,  6.3522e-01,  3.2069e-01,  ..., -1.9514e-01,
         -5.5015e-01, -8.4486e-01],
        [-3.6200e-01,  7.3842e-02,  2.7510e-03,  ..., -4.0192e-01,
         -2.7595e-01, -6.8949e-01],
        ...,
        [-4.7121e-01, -1.4553e+00, -8.0372e-01,  ..., -1.4037e+00,
          9.6659e-02, -9.1518e-02],
        [ 6.8689e-01,  2.4330e+00,  5.4247e-01,  ..., -4.9704e-03,
          8.0625e-01,  3.5779e+00],
        [-1.9339e-01, -4.4942e-01, -3.5243e-01,  ..., -1.3916e-01,
         -1.9402e+00,  6.9739e-01]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

It has parallel processing architecture that allows it to perform multiple calculations at the same time

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

the available memory of a computer hardware.